# Latency Lab Analysis

This notebook reads `results/index.csv` and shows basic comparisons.


In [ ]:
import sys
from pathlib import Path

def find_repo_root(start: Path) -> Path:
    for parent in [start] + list(start.parents):
        if (parent / "scripts" / "results_lib.py").exists():
            return parent
    return start

repo_root = find_repo_root(Path().resolve())
sys.path.append(str(repo_root / "scripts"))

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from results_lib import load_index, load_samples

sns.set_theme(style="whitegrid")


## Run benchmarks from the notebook

Use this helper to launch a benchmark and print the latest stdout summary.


In [ ]:
import sys
from pathlib import Path

import ipywidgets as widgets
from IPython.display import display, clear_output

def find_repo_root(start: Path) -> Path:
    for parent in [start] + list(start.parents):
        if (parent / "scripts" / "notebook_runner.py").exists():
            return parent
    return start

repo_root = find_repo_root(Path().resolve())
scripts_dir = repo_root / "scripts"
if str(scripts_dir) not in sys.path:
    sys.path.append(str(scripts_dir))

import notebook_runner as nb

bench_path = repo_root / "build" / "bench"
results_dir = repo_root / "results"

try:
    cases = nb.list_cases(bench_path)
except Exception as exc:
    cases = ["noop"]
    print(f"Could not list cases: {exc}")
    print(f"Build bench at {bench_path} to populate the list.")

DEFAULT_ITERS = 10000
# Warmup iterations are discarded before measurement to stabilize caches, CPU state, etc.
DEFAULT_WARMUP = 1000
DEFAULT_PIN_CPU = 0

CHECKBOX_WIDTH = "28px"
CASE_WIDTH = "180px"
ITERS_WIDTH = "120px"
WARMUP_WIDTH = "120px"
AFFINITY_WIDTH = "190px"
PIN_WIDTH = "110px"
INPUT_STYLE = {"description_width": "0px"}

case_rows = {}


def make_affinity_group(label: str, value: bool):
    checkbox = widgets.Checkbox(
        value=value,
        description="",
        indent=False,
        layout=widgets.Layout(width="18px", margin="0px"),
    )
    text = widgets.Label(
        value=label,
        layout=widgets.Layout(margin="0px 6px 0px 0px"),
    )
    group = widgets.HBox(
        [checkbox, text],
        layout=widgets.Layout(align_items="center", gap="2px"),
    )
    return checkbox, group


def make_case_row(name: str) -> dict:
    enabled = widgets.Checkbox(
        value=False,
        description="",
        layout=widgets.Layout(width=CHECKBOX_WIDTH),
    )
    case_label = widgets.Label(
        value=name,
        layout=widgets.Layout(width=CASE_WIDTH),
    )
    iters = widgets.IntText(
        value=DEFAULT_ITERS,
        description="",
        layout=widgets.Layout(width=ITERS_WIDTH),
        style=INPUT_STYLE,
    )
    warmup = widgets.IntText(
        value=DEFAULT_WARMUP,
        description="",
        layout=widgets.Layout(width=WARMUP_WIDTH),
        style=INPUT_STYLE,
    )

    affinity_unpinned, unpinned_group = make_affinity_group("unpinned", True)
    affinity_pinned, pinned_group = make_affinity_group("pinned", False)
    affinity_box = widgets.HBox(
        [unpinned_group, pinned_group],
        layout=widgets.Layout(
            width=AFFINITY_WIDTH,
            justify_content="flex-start",
            align_items="center",
            gap="4px",
            overflow="visible",
        ),
    )

    pin_cpu = widgets.IntText(
        value=DEFAULT_PIN_CPU,
        description="",
        layout=widgets.Layout(width=PIN_WIDTH),
        style=INPUT_STYLE,
    )

    def sync_affinity(_=None):
        needs_pin = affinity_pinned.value
        pin_cpu.disabled = not needs_pin
        if needs_pin and pin_cpu.value < 0:
            pin_cpu.value = 0

    affinity_pinned.observe(sync_affinity, names="value")
    sync_affinity()

    row = widgets.HBox(
        [enabled, case_label, iters, warmup, affinity_box, pin_cpu],
        layout=widgets.Layout(align_items="center", overflow="visible"),
    )
    return {
        "name": name,
        "row": row,
        "enabled": enabled,
        "iters": iters,
        "warmup": warmup,
        "affinity_unpinned": affinity_unpinned,
        "affinity_pinned": affinity_pinned,
        "pin_cpu": pin_cpu,
    }


case_rows = {name: make_case_row(name) for name in cases}
rows_box = widgets.VBox(
    [entry["row"] for entry in case_rows.values()],
    layout=widgets.Layout(max_height="260px", overflow_y="auto", overflow_x="hidden"),
)

cases_header = widgets.HBox(
    [
        widgets.HTML("&nbsp;", layout=widgets.Layout(width=CHECKBOX_WIDTH)),
        widgets.HTML("<b>Case</b>", layout=widgets.Layout(width=CASE_WIDTH)),
        widgets.HTML("<b>Iters</b>", layout=widgets.Layout(width=ITERS_WIDTH)),
        widgets.HTML("<b>Warmup</b>", layout=widgets.Layout(width=WARMUP_WIDTH)),
        widgets.HTML(
            "<b>Affinity</b>", layout=widgets.Layout(width=AFFINITY_WIDTH)
        ),
        widgets.HTML("<b>Pin CPU</b>", layout=widgets.Layout(width=PIN_WIDTH)),
    ],
    layout=widgets.Layout(align_items="center"),
)

filter_text = widgets.Text(value="", description="Filter")


def apply_filter(_=None):
    query = filter_text.value.strip().lower()
    if not query:
        visible = [entry["row"] for entry in case_rows.values()]
    else:
        visible = [
            entry["row"]
            for name, entry in case_rows.items()
            if query in name.lower()
        ]
    rows_box.children = visible


filter_text.observe(apply_filter, names="value")
apply_filter()

lab = widgets.Text(value="os", description="Lab")
tags = widgets.Text(value="quiet", description="Tags (csv)")
tags_help = widgets.HTML(
    "Tags are free-form labels used only for naming runs and filtering results."
)

update_mode = widgets.RadioButtons(
    options=[
        ("Append (default)", "append"),
        ("Skip summary/index", "skip"),
        ("Replace matching in index", "replace"),
    ],
    value="append",
    description="Update",
)

warmup_help = widgets.HTML(
    "Warmup runs are discarded before measurement to stabilize CPU/cache state."
)

select_all = widgets.Button(description="Select all")
deselect_all = widgets.Button(description="Deselect all")
run_button = widgets.Button(description="Run selected")
output = widgets.Output()


def select_all_clicked(_):
    for entry in case_rows.values():
        entry["enabled"].value = True


def deselect_all_clicked(_):
    for entry in case_rows.values():
        entry["enabled"].value = False


select_all.on_click(select_all_clicked)
deselect_all.on_click(deselect_all_clicked)


def affinity_variants(unpinned: bool, pinned: bool, pin_value: int):
    variants = []
    if unpinned:
        variants.append(("unpinned", None))
    if pinned:
        variants.append((f"pinned cpu {pin_value}", pin_value))
    return variants


def run_clicked(_):
    with output:
        clear_output()
        selected = [
            entry for entry in case_rows.values() if entry["enabled"].value
        ]
        if not selected:
            print("No cases selected.")
            return
        tag_list = [t.strip() for t in tags.value.split(",") if t.strip()]

        total_runs = 0
        for entry in selected:
            total_runs += len(
                affinity_variants(
                    entry["affinity_unpinned"].value,
                    entry["affinity_pinned"].value,
                    entry["pin_cpu"].value,
                )
            )

        run_idx = 0
        for entry in selected:
            case_name = entry["name"]
            iters_value = entry["iters"].value
            warmup_value = entry["warmup"].value
            unpinned = entry["affinity_unpinned"].value
            pinned = entry["affinity_pinned"].value
            pin_value = entry["pin_cpu"].value
            if pinned and pin_value < 0:
                print(
                    f"{case_name}: Pin CPU must be >= 0 when affinity includes pinned."
                )
                continue
            variants = affinity_variants(unpinned, pinned, pin_value)
            if not variants:
                print(f"{case_name}: Select at least one affinity option.")
                continue
            for label, pin in variants:
                run_idx += 1
                suffix = f" ({label})" if len(variants) > 1 else ""
                print(f"[{run_idx}/{total_runs}] {case_name}{suffix}")
                try:
                    result = nb.run_case(
                        bench_path=bench_path,
                        lab=lab.value,
                        case=case_name,
                        results_dir=results_dir,
                        iters=iters_value,
                        warmup=warmup_value,
                        pin_cpu=pin,
                        tags=tag_list,
                        update_mode=update_mode.value,
                    )
                except Exception as exc:
                    print(f"Run failed for {case_name}: {exc}")
                    continue
                print(f"Run dir: {result.run_dir}")
                print(result.read_stdout())


run_button.on_click(run_clicked)


In [ ]:
display(
    widgets.VBox(
        [
            filter_text,
            widgets.HBox([select_all, deselect_all, run_button]),
            cases_header,
            rows_box,
            warmup_help,
            lab,
            tags,
            tags_help,
            update_mode,
        ]
    ),
    output,
)


In [ ]:
index = load_index(repo_root / "results" / "index.csv")

if hasattr(index, "columns") and "run_dir" in index.columns:
    def _abs_run_dir(value):
        if not isinstance(value, str) or not value:
            return value
        path = Path(value)
        if path.is_absolute():
            return value
        return str((repo_root / path).resolve())

    index["run_dir"] = index["run_dir"].apply(_abs_run_dir)

index.head()


In [ ]:
# Pick a case to compare pinned vs unpinned (adjust as needed)
case_name = index["case"].iloc[0] if len(index) else "noop"
subset = index[index["case"] == case_name]

display(subset[["case","tags","pin_cpu","p50","p95","p99","p999","mean"]])


In [ ]:
# Long-form view of quantiles for the chosen case
if len(subset):
    plot_df = subset.melt(
        id_vars=["tags","pin_cpu"],
        value_vars=["p50","p95","p99","p999"],
        var_name="quantile",
        value_name="ns",
    )
    plt.figure(figsize=(8,4))
    sns.barplot(data=plot_df, x="quantile", y="ns", hue="pin_cpu")
    plt.title(f"{case_name} quantiles by pin_cpu")
    plt.show()


In [ ]:
# ECDF of raw samples for the first two runs (if raw data is present)
def short_label(value):
    if not isinstance(value, str):
        return str(value)
    path = Path(value)
    parts = path.parts
    if "results" in parts:
        idx = parts.index("results")
        tail = parts[idx + 1 :]
        if len(tail) >= 3:
            return "/".join(tail[:3])
        if tail:
            return "/".join(tail)
    try:
        return str(path.relative_to(repo_root))
    except Exception:
        return path.name or str(value)

if len(index) >= 2:
    run_a = index.iloc[0]["run_dir"]
    run_b = index.iloc[1]["run_dir"]
    samples_a = load_samples(run_a)
    samples_b = load_samples(run_b)

    plt.figure(figsize=(12,8))
    sns.ecdfplot(samples_a, label=short_label(run_a))
    sns.ecdfplot(samples_b, label=short_label(run_b))
    plt.xlabel("ns")
    plt.ylabel("ECDF")
    plt.legend()
    plt.title("ECDF comparison")
    plt.show()
